**Applied Statistcs Project 2 - Data Cleaning in Python**

In [9]:
import numpy as np
import pandas as pd

In [10]:
df = pd.read_csv('used_cars_data.csv', nrows=3000, usecols=[
    'body_type',
    'daysonmarket',
    'fuel_type',
    'has_accidents',
    'highway_fuel_economy',
    'horsepower',
    'is_new',
    'make_name',
    'mileage',
    'price',
    'seller_rating',
    'year'
])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body_type             2995 non-null   object 
 1   daysonmarket          3000 non-null   int64  
 2   fuel_type             2953 non-null   object 
 3   has_accidents         2124 non-null   object 
 4   highway_fuel_economy  2397 non-null   float64
 5   horsepower            2920 non-null   float64
 6   is_new                3000 non-null   bool   
 7   make_name             3000 non-null   object 
 8   mileage               2751 non-null   float64
 9   price                 3000 non-null   float64
 10  seller_rating         2997 non-null   float64
 11  year                  3000 non-null   int64  
dtypes: bool(1), float64(5), int64(2), object(4)
memory usage: 260.9+ KB


In [11]:
df['body_type'] = df['body_type'].fillna(df['body_type'].mode()[0]) #fill with the most frequent value
df['mileage'] = df['mileage'].fillna(df['mileage'].mean()) #fill with the mean


#Missing values in highway_fuel_economy are filled based on the median value for each body_type
hfe = df.groupby(['body_type'])['highway_fuel_economy'].median().to_dict()
df['highway_fuel_economy'] = df['highway_fuel_economy'].fillna(df['body_type'].map(hfe))



#High-NaN columns (more than 40%) benefit from a common value like the mode, as imputing with the mean in such columns could introduce bias and 
#skews results
nullsrs = (df.isnull().mean() * 100).sort_values(ascending=False)
lst = nullsrs.loc[nullsrs > 40].index.to_list()
for col in lst:
    df[col] = df[col].fillna(df[col].mode().values[0])
    
#Low-Nan columns (less than 7.5%) can tolerate mean imputation for numeric data, as only a few entries are missing, so the data distribution 
#remains largely unaffected
nullsrs = (df.isnull().mean()*100).sort_values(ascending = False)
del_lst = nullsrs.loc[nullsrs < 7.5].index.to_list()
for col in df[del_lst].select_dtypes(['object']).columns.to_list():
  df[col] = df[col].fillna(df[col].mode()[0])
for col in df[del_lst].select_dtypes(['int64' , 'float64']).columns.to_list():
  df[col] = df[col].fillna(df[col].mean())
  


In [12]:
df.head()

,body_type,daysonmarket,fuel_type,has_accidents,highway_fuel_economy,horsepower,is_new,make_name,mileage,price,seller_rating,year
0,SUV / Crossover,522,Gasoline,NaN,26.0,177.0,True,Jeep,7.000000,23141.0,2.800000,2019
1,SUV / Crossover,207,Gasoline,NaN,26.0,246.0,True,Land Rover,8.000000,46500.0,3.000000,2020
2,Sedan,1233,Gasoline,False,23.0,305.0,False,Subaru,47846.749546,46995.0,3.793942,2016
3,SUV / Crossover,196,Gasoline,NaN,26.0,340.0,True,Land Rover,11.000000,67430.0,3.000000,2020
4,SUV / Crossover,137,Gasoline,NaN,26.0,246.0,True,Land Rover,7.000000,48880.0,3.000000,2020


In [13]:

# Save the cleaned DataFrame to a new CSV file
cleaned_file_path = 'cleaned_used_cars_data.csv'  
df.to_csv(cleaned_file_path, index=False)